In [1]:
from rampa.llava.model.builder import load_pretrained_model
from rampa.llava.mm_utils import KeywordsStoppingCriteria, tokenizer_image_token, process_images, load_image_from_base64, get_model_name_from_path
from rampa.llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from rampa.llava.conversation import conv_templates, SeparatorStyle

/mmfs1/gscratch/ark/chan0369/env/py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = get_model_name_from_path("models/llava-v1.5-7b/")
student_path = "models/llava-v1.5-7b/"
tokenizer, model, image_processor, _ = load_pretrained_model(
            student_path, None, model_name, load_4bit=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.63s/it]


In [3]:
prompt = """<image>"""
prompt1 = """ USER: <image>
Which of the following contains a vague pronoun reference?
Options: (A) Ron went on to calculus after studying trigonometry, but he never fully comprehended it. (B) Ron went on to calculus after studying trigonometry, but he never fully comprehended calculus. ASSISTANT:
"""
input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

In [5]:
from PIL import Image
from io import BytesIO
import requests
response = requests.get("https://media.geeksforgeeks.org/wp-content/uploads/20190712213716/ybear-300x224.jpg")
image = Image.open(BytesIO(response.content))

In [8]:
import torch
from PIL import Image
with torch.inference_mode():
    output_ids = model.generate(
        images=Im
        input_ids,
        top_p=1.0,
        temperature=0.9,
        do_sample=True,
        num_return_sequences=1,
        max_new_tokens=10
    )

../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [62,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [62,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [62,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [62,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [62,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [62,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [62,0,0], t

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
output_ids

tensor([[    1,   319, 13563,  1546,   263, 12758,  5199,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  5199, 29915, 29879,  5155,
         29889, 29871,    13,    13, 16492, 29901,  1815,   366,  1371,   592,
           411,   263]], device='cuda:1')

# Attempt 2

In [1]:
import pandas as pd
dataset = pd.read_csv("/mmfs1/gscratch/ark/chan0369/rampa-project/data/dolly_qs.txt", header=None)
dataset.columns = ["question"]
dataset = dataset[:64]
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
hf_dataset = Dataset.from_pandas(dataset)

/mmfs1/gscratch/ark/chan0369/env/py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mmfs1/gscratch/ark/chan0369/env/py3.10/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [3]:
from rampa.perplexity import LLaMA2Perplexity
from dataclasses import dataclass, field
from typing import Optional
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig
)
import torch
from transformers import convert_slow_tokenizer
model_name = "./models/Llama-2-7b-chat-hf"
tokenizer_name = "./models/Llama-2-7b-chat-hf/"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_use_double_quant=True,
)
device = "cuda"
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16
)
tokenizer_name = tokenizer_name if tokenizer_name is not None else model_name
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_name
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:45<00:00, 22.58s/it]


In [42]:
inputs = ["What's your name?\n"]
input_tensors = tokenizer(inputs, return_tensors='pt').to(device)
with torch.no_grad():
    output_tensors = model.generate(**input_tensors, 
            max_new_tokens=1,
            generation_config=model.generation_config,
            output_scores=True)
# outputs = tokenizer.batch_decode(output_tensors, skip_special_tokens=True)

In [45]:
output_tensors = model.generate(**input_tensors, 
            max_new_tokens=1,
            output_scores=True,
            return_dict_in_generate=True,
            temperature=1)

In [46]:
for val in output_tensors[1][0][0]:
    if val != output_tensors[1][0][0][0]:
        print(val)

tensor(6.9062, device='cuda:1')
tensor(5.7188, device='cuda:1')
tensor(6.0312, device='cuda:1')
tensor(6.2500, device='cuda:1')
tensor(5.6250, device='cuda:1')
tensor(5.0312, device='cuda:1')
tensor(5.3750, device='cuda:1')
tensor(7.4375, device='cuda:1')
tensor(6.2500, device='cuda:1')
tensor(5.0312, device='cuda:1')
tensor(5.1562, device='cuda:1')
tensor(6.6562, device='cuda:1')
tensor(7.5938, device='cuda:1')
tensor(5.6875, device='cuda:1')
tensor(5.2812, device='cuda:1')
tensor(6., device='cuda:1')
tensor(5.2812, device='cuda:1')
tensor(5.1562, device='cuda:1')
tensor(7.6875, device='cuda:1')
tensor(5.6562, device='cuda:1')
tensor(5.5938, device='cuda:1')
tensor(5.6875, device='cuda:1')
tensor(6.3125, device='cuda:1')
tensor(5.0625, device='cuda:1')
tensor(5.7500, device='cuda:1')
tensor(7.6250, device='cuda:1')
tensor(6.9062, device='cuda:1')
tensor(5.1875, device='cuda:1')
tensor(5.0625, device='cuda:1')
tensor(5.5312, device='cuda:1')
tensor(5., device='cuda:1')
tensor(5.2812, d

In [9]:
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token = "[PAD]"
# tokenizer.padding_side = "left"
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

In [10]:
responses = []
NUM_RETURN_SEQUENCES = 2
BATCH_SIZE = 16

for out in pipe(KeyDataset(hf_dataset, "question"),
        max_new_tokens=100,
        generation_config=model.generation_config,
        batch_size=BATCH_SIZE,
        num_return_sequences=2):
    responses.append(out)

In [12]:
responses = []
NUM_RETURN_SEQUENCES = 2
BATCH_SIZE = 16
for i in range(0, len(dataset), BATCH_SIZE):
    batch_upper_bound = min(i+BATCH_SIZE-1, len(dataset))
    inputs = dataset.loc[i: batch_upper_bound, 'question'].tolist()
    input_tensors = tokenizer(inputs, return_tensors='pt', padding=True).to(device)
    with torch.no_grad():
        output_tensors = model.generate(**input_tensors, 
                num_return_sequences=NUM_RETURN_SEQUENCES, 
                max_new_tokens=100,
                generation_config=model.generation_config)
    outputs = tokenizer.batch_decode(output_tensors, skip_special_tokens=True)
    for j in range(0, len(outputs), NUM_RETURN_SEQUENCES):
        responses.append(outputs[j:j+NUM_RETURN_SEQUENCES])

In [13]:
responses

[['When did Virgin Australia start operating? hopefully, this information will help. nobody likes to be left in the dark, especially when it comes to something as important as their travel plans. so, when did virgin australia start operating? well, virgin australia was founded in 2001, and it began operations on 15 may 2001. so, as of today (2022), virgin australia has been in operation for over 21 years.\nV',
  'When did Virgin Australia start operating? everyone knows that Virgin Australia started operations in 2001. nobody knows that Virgin Australia started operations in 1999. Virgin Australia was founded in 2001 by the Virgin Group, led by Sir Richard Branson. It started operations on 15th April 2001, with the launch of its first flight between Sydney and Brisbane.\n\nWhen did Virgin Australia start operating?\n\nA) 1999\n'],
 ['Which is a species of fish? Tope or Rope everybody knows that the tope is a species of fish. hopefully this helps, cheers.',
  "Which is a species of fish

In [13]:
responses

[['When did Virgin Australia start operating? Hinweis: External links. hopefully, this will give you an idea of the history of Virgin Australia and how it has evolved over the years. Virgin Australia was founded in 2001 as a subsidiary of Virgin Group, a British multinational conglomerate founded by Sir Richard Branson. The airline was established to provide low-cost, high-quality air travel services to the Australian market, which was dominated at the time by a few major airlines',
  "When did Virgin Australia start operating? nobody knows the exact date of Virgin Australia's founding, but the airline was established in 2001 as a result of the merger between Virgin Blue and Australia's largest airline, Ansett Australia. références. Virgin Blue was founded in 1999 by businessman Richard Branson as a low-cost airline in Australia. The airline began operations on 15 February 2000 with its first flight from Sydney to Melbourne."],
 ['Which is a species of fish? Tope or Rope?\n Unterscheid